# Data Analysis
This file will contains all initial analysis of the data and plot it.

In [13]:
import numpy as np
import pandas as pd
import nltk
from nltk import FreqDist
from config import processed_dir_path, token_youtube_link, token_internet_link, token_emoticon_funny
from nltk.corpus import stopwords
set(stopwords.words('english'))

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

### calculate_most_common_words
Find the most recurrent words for each subreddit. The comments are cleaned beforehand (stopwords and punctuation removed)

In [14]:
def calculate_most_common_words(data, top_nb = 5):
    print("Computing most common words...")
    
    fdist_allsubs = FreqDist()
    list_subreddit_freq = []
    subreddit_names = data.subreddits.unique()
    
    for subreddit in subreddit_names:
        print("\t" + subreddit)
        
        # get data associated to subreddit
        subreddit_data = data.loc[data["subreddits"] == subreddit]
        
        # tokenize
        subreddit_comments = nltk.tokenize.word_tokenize(subreddit_data["comments"].str.cat(sep="\n"))
        # remove stopswords and punctuation
        subreddit_comments = [word for word in subreddit_comments if (word not in stopwords.words('english') and word.isalpha())]
        
        # get frequency distribution of all words
        fdist_subreddit = nltk.FreqDist(subreddit_comments)
        
        fdist_subreddit.plot(5, cumulative = False)
        plt.show()
        
        print(fdist_subreddit)
        print(fdist_subreddit.most_common(top_nb))
        list_subreddit_freq.append(fdist_subreddit)
        fdist_allsubs += fdist_subreddit
    
    print("\tall subreddits")
    print(fdist_allsubs)
    print(fdist_allsubs.most_common(top_nb))
    
    i = 0
    for subreddit_freq in list_subreddit_freq:
        idtf(fdist_allsubs, subreddit_freq, subreddit_names[i])
        i+=1

### IDTF
Using the frequency distribution we already have, find occurence of word in on subreddit / occurence of word in all

In [15]:
def idtf(fdist_all, fdist_subreddit, subreddit_name):
    words = fdist_all.keys()
    dictionary = {}
    for word in words:
        occurence_subreddit = fdist_subreddit[word]/fdist_all[word]
        if (occurence_subreddit > 0.5):
            dictionary[word] = occurence_subreddit
#             print("The word " + word + " occurs quite often in " + subreddit_name)
    sorted_dic = sorted(dictionary.items(), key=lambda kv: kv[1])
    print("The top 10 words for " + subreddit_name + ": \n\t")
    print(sorted_dic[-10:])

### calculate_avg_nb_words
Compute average amount of words per comment, and also prints the total number of youtube links in the entire subreddit.

In [16]:
def calculate_avg_nb_words(data):
    COUNT_COLUMN = "count words"
    
    print("Computing average number of words per comment...")
    data[COUNT_COLUMN] = data.apply(lambda row: len(row.comments), axis=1)
    count_youtubelink, count_internetlink, count_emoticon = 0,0,0
    
    for subreddit in data.subreddits.unique():
        print("\n\n\t" + subreddit)
        # get data associated to subreddit
        subreddit_data = data.loc[data["subreddits"] == subreddit]
        
        # get mean of count of words 
        subreddit_mean_words = subreddit_data[COUNT_COLUMN].mean()
        print("Average words/comment:"+ str(subreddit_mean_words))
        
        # find the nb of youtube links
        subreddit_count_youtubelink = subreddit_data["comments"].str.count(token_youtube_link).sum()
        print("nb of youtube links in " + subreddit + ":" + str(subreddit_count_youtubelink))
        #add to global youtube count
        count_youtubelink += subreddit_count_youtubelink
        
        # find the nb of internet links
        subreddit_count_internetlink = subreddit_data["comments"].str.count(token_internet_link).sum()
        print("nb of internet links in " + subreddit + ":" + str(subreddit_count_internetlink))
        #add to global youtube count
        count_internetlink += subreddit_count_internetlink
        
        # find the nb of emoticons
        subreddit_count_emoticon = subreddit_data["comments"].str.count(token_emoticon_funny).sum()
        print("nb of emoticons in" + subreddit + ":" + str(subreddit_count_emoticon))
        #add to global youtube count
        count_emoticon += subreddit_count_emoticon
        
    print("\n\n\t all subreddits")
    # get mean of count of words 
    mean_words = data[COUNT_COLUMN].mean()
    print("Average words/comment:" + srt(mean_words))
    print("Total nb of youtube links:" + str(count_youtubelink))
    print("Total nb of internet links:" + str(count_internetlink))
    print("Total nb of emoticons:" + str(count_emoticon))

# MAIN

In [17]:
file_path = processed_dir_path + "/LEMMA_train_clean.csv"
print("Reading data file...")
data = pd.read_csv(file_path)

print("Computing statistics...") 
calculate_most_common_words(data)
calculate_avg_nb_words(data)


#     TODO: scoring function

Reading data file...
Computing statistics...
Computing most common words...
	hockey


<Figure size 640x480 with 1 Axes>

NameError: name 'plt' is not defined